In [1]:
#import packages and functions

%load_ext autoreload
%autoreload 2

import pursuit_functions as pursuit
    
import pandas as pd
import numpy as np
import polars as pl
from itertools import product
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm 
from numba import njit

from scipy import stats
from sklearn.decomposition import PCA
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
#load data set

#all_pursuit_tasks = pd.read_parquet("ca1_ca3_rsc_pursuit_data.parquet", engine="pyarrow")

In [4]:
#load data sets

RSC_tasks = pd.read_parquet("RSC_sessions.parquet", engine="pyarrow")
PPC_tasks = pd.read_parquet("PPC_sessions.parquet", engine="pyarrow")

# Normalize points and find circle boundaries.

In [5]:
#get all coordinate values below 99th percentile and normalize points for all regions 

#normalized_sessions = pursuit.tuning.normalize_points(all_pursuit_tasks)
RSC_norm_sessions = pursuit.tuning.normalize_points(RSC_tasks)
PPC_norm_sessions = pursuit.tuning.normalize_points(PPC_tasks)

In [6]:
#find the mean center and overall radius of the arena for all normalized data points
#you can specify the percentile value to be considered for the overall radius; default is 95th percentile
#calculates the individual center point for each session

#circle_boundaries, radius = pursuit.tuning.fit_circle_bounds(normalized_sessions)
#print(radius)

RSC_circle_boundaries, RSC_radius = pursuit.tuning.fit_circle_bounds(RSC_norm_sessions)
PPC_circle_boundaries, PPC_radius = pursuit.tuning.fit_circle_bounds(PPC_norm_sessions)
print(RSC_radius)
print(PPC_radius)

60.59943392406157
60.94173773808301


In [7]:
#find circumference points for plotting using the center coordinates and overall radius
#all_circ_points = pursuit.tuning.circumference(circle_boundaries)

RSC_circ_points = pursuit.tuning.circumference(RSC_circle_boundaries)
PPC_circ_points = pursuit.tuning.circumference(PPC_circle_boundaries)

# Plot the laser coordinates and boundaries.

In [ ]:
#plot normalized concatenated laser and rat paths with center point and boundary
#the function takes the normalized_sessions, circle_boundaries, and all_circ_points dataframes

#pursuit.tuning.plot_arena_bounds(normalized_sessions, circle_boundaries, all_circ_points)

#pursuit.tuning.plot_arena_bounds(RSC_norm_sessions, RSC_circle_boundaries, RSC_circ_points)

pursuit.tuning.plot_arena_bounds(PPC_norm_sessions, PPC_circle_boundaries, PPC_circ_points)

# Clean data and pull spike data.

In [ ]:
# obtain region-specific sessions
#RSC_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "RSC"]
#CA1_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "CA1"]
#CA3_sessions = all_pursuit_tasks[all_pursuit_tasks["region"] == "CA3"]



In [8]:
#obtain trial block-specific sessions

#RSC_pursuit = RSC_sessions[RSC_sessions["trial_block"] == "pursuit"]
#CA1_pursuit = CA1_sessions[CA1_sessions["trial_block"] == "pursuit"]
#CA3_pursuit = CA3_sessions[CA3_sessions["trial_block"] == "pursuit"]

RSC_pursuit = RSC_tasks[RSC_tasks["block"] == "pursuit"]
PPC_pursuit = PPC_tasks[PPC_tasks["block"] == "pursuit"]

In [9]:
#drop NA values for RSC, CA1, and CA3 sessions

#RSC_cleaned = pursuit.tuning.drop_NA_vals(RSC_pursuit)
#CA1_cleaned = pursuit.tuning.drop_NA_vals(CA1_pursuit)
#CA3_cleaned = pursuit.tuning.drop_NA_vals(CA3_pursuit)

RSC_cleaned = pursuit.tuning.drop_NA_vals(RSC_pursuit)
PPC_cleaned = pursuit.tuning.drop_NA_vals(PPC_pursuit)

In [ ]:
PPC_cleaned

In [28]:
len(PPC_cleaned[PPC_cleaned["sessFile"] == "KB20_05_pursuitRoot.mat"])

83986

In [16]:
PPC_cleaned[PPC_cleaned["sessFile"] == "KB20_05_pursuitRoot.mat"]["spkTable_2"].values.unique()

<IntegerArray>
[0, 1, 3, 2, 4, 5, 6, 8, 7]
Length: 9, dtype: Int16

# Normalizing time for bootstrapping.

In [10]:
#RSC_clean_time = pursuit.tuning.normalize_time(RSC_cleaned)
#CA1_clean_time = pursuit.tuning.normalize_time(CA1_cleaned)
#CA3_clean_time = pursuit.tuning.normalize_time(CA3_cleaned)

RSC_clean_time = pursuit.tuning.normalize_time(RSC_cleaned)
PPC_clean_time = pursuit.tuning.normalize_time(PPC_cleaned)

/Users/may/Projects/pursuit/pursuit_functions/tuning.py:184: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["rel_time"] = df.groupby("sessFile")["time"].transform(lambda x: x - x.min())
/Users/may/Projects/pursuit/pursuit_functions/tuning.py:184: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df["rel_time"] = df.groupby("sessFile")["time"].transform(lambda x: x - x.min())


# Assigning epochs to dataframes with normalized time.

In [33]:
# for epoch_half, True:1, False:2 will result in the first half labeled as 1 (<= cutoff) and the second half labeled as 2
# for epoch_odd_even, we take the minutes divided by 2 and find the remainder. If the remainder is 1 (odd), it will be labeled as 1 and if the remainder is 0 (even), it will be labeled as 2

#RSC_clean_time_epochs = pursuit.tuning.assign_epochs(RSC_clean_time)
#CA1_clean_time_epochs = pursuit.tuning.assign_epochs(CA1_clean_time)
#CA3_clean_time_epochs = pursuit.tuning.assign_epochs(CA3_clean_time)

#RSC_clean_time_epochs = pursuit.tuning.assign_epochs(RSC_clean_time)
PPC_clean_time_epochs = pursuit.tuning.assign_epochs(PPC_clean_time)

<string>:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.


In [36]:
count_neurons(PPC_clean_time_epochs)

({'KB09_12_pursuitRoot.mat': 3,
  'KB09_13_pursuitRoot.mat': 2,
  'KB09_17_pursuitRoot.mat': 6,
  'KB09_18_pursuitRoot.mat': 11,
  'KB09_19_pursuitRoot.mat': 6,
  'KB09_20_pursuitRoot.mat': 1,
  'KB09_22_pursuitRoot.mat': 1,
  'KB09_23_pursuitRoot.mat': 1,
  'KB09_24_pursuitRoot.mat': 1,
  'KB09_25_pursuitRoot.mat': 1,
  'KB09_26_pursuitRoot.mat': 1,
  'KB10_05_pursuitRoot.mat': 3,
  'KB10_07_pursuitRoot.mat': 1,
  'KB10_08_pursuitRoot.mat': 4,
  'KB19_03_pursuitRoot.mat': 4,
  'KB19_04_pursuitRoot.mat': 6,
  'KB19_05_pursuitRoot.mat': 9,
  'KB19_07_pursuitRoot.mat': 13,
  'KB19_08_pursuitRoot.mat': 13,
  'KB19_14_pursuitRoot.mat': 6,
  'KB19_15_pursuitRoot.mat': 9,
  'KB19_16_pursuitRoot.mat': 11,
  'KB19_17_pursuitRoot.mat': 10,
  'KB19_18_pursuitRoot.mat': 10,
  'KB19_19_pursuitRoot.mat': 10,
  'KB19_20_pursuitRoot.mat': 10,
  'KB19_21_pursuitRoot.mat': 10,
  'KB19_22_pursuitRoot.mat': 8,
  'KB19_23_pursuitRoot.mat': 8,
  'KB19_24_pursuitRoot.mat': 6,
  'KB19_25_pursuitRoot.mat': 8,

# Create concise dataframes with only sessFile, laser, epoch, spike, and relative time data.

In [11]:
#make a new df with only sessFile, laser, epoch, spike, and relative time data.

#RSC_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(RSC_clean_time_epochs)
#CA1_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(CA1_clean_time_epochs)
#CA3_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(CA3_clean_time_epochs)

RSC_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(RSC_clean_time_epochs)
PPC_epoch_laser_spks = pursuit.tuning.epoch_laser_spks(PPC_clean_time_epochs)

# Pull epoch halves into two data frames for tuning correlation.

In [12]:
#make separate dfs for df subsets used for tuning

RSC_epoch_first_half, RSC_epoch_second_half, RSC_epoch_odd_min, RSC_epoch_even_min = pursuit.tuning.pull_epochs(RSC_epoch_laser_spks)
PPC_epoch_first_half, PPC_epoch_second_half, PPC_epoch_odd_min, PPC_epoch_even_min = pursuit.tuning.pull_epochs(PPC_epoch_laser_spks)

#CA1_epoch_first_half, CA1_epoch_second_half, CA1_epoch_odd_min, CA1_epoch_even_min = pursuit.tuning.pull_epochs(CA1_epoch_laser_spks)
#CA3_epoch_first_half, CA3_epoch_second_half, CA3_epoch_odd_min, CA3_epoch_even_min = pursuit.tuning.pull_epochs(CA3_epoch_laser_spks)

In [30]:
len(PPC_epoch_second_half[PPC_epoch_second_half["sessFile"] == "KB20_05_pursuitRoot.mat"])

30786

In [29]:
len(PPC_epoch_first_half[PPC_epoch_first_half["sessFile"] == "KB20_05_pursuitRoot.mat"])

51528

# Bootstrapping.

In [13]:
#bootstrap sessions
RSC_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(RSC_epoch_first_half, RSC_epoch_second_half, RSC_circle_boundaries)
RSC_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(RSC_epoch_odd_min, RSC_epoch_even_min, RSC_circle_boundaries)

PPC_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(PPC_epoch_first_half, PPC_epoch_second_half, PPC_circle_boundaries)
PPC_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(PPC_epoch_odd_min, PPC_epoch_even_min, PPC_circle_boundaries)

#CA1_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(CA1_epoch_first_half, CA1_epoch_second_half, circle_boundaries)
#CA1_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(CA1_epoch_odd_min, CA1_epoch_even_min, circle_boundaries)

#CA3_first_second_tuning = pursuit.tuning.bootstrap_all_sessions(CA3_epoch_first_half, CA3_epoch_second_half, circle_boundaries)
#CA3_odd_even_tuning = pursuit.tuning.bootstrap_all_sessions(CA3_epoch_odd_min, CA3_epoch_even_min, circle_boundaries)

Sessions: 100%|██████████| 74/74 [01:26<00:00,  1.17s/it]


In [35]:
RSC_first_second_tuning

,sessFile,neuron,bootstrap_iter,spearman_r,p_val
0,KB10_01_pursuitRoot.mat,spkTable_1,0,0.060150,0.801113
1,KB10_01_pursuitRoot.mat,spkTable_1,1,-0.243609,0.300668
2,KB10_01_pursuitRoot.mat,spkTable_1,2,0.007519,0.974903
3,KB10_01_pursuitRoot.mat,spkTable_1,3,-0.263158,0.262287
4,KB10_01_pursuitRoot.mat,spkTable_1,4,-0.347368,0.133449
...,...,...,...,...,...
131083,LP06_29_pursuitRoot.mat,spkTable_8,933,0.157637,0.506849
131084,LP06_29_pursuitRoot.mat,spkTable_9,933,-0.039112,0.869955
131085,LP06_29_pursuitRoot.mat,spkTable_7,934,-0.140060,0.555893
131086,LP06_29_pursuitRoot.mat,spkTable_8,934,-0.040678,0.864795


In [34]:
RSC_odd_even_tuning

,sessFile,neuron,bootstrap_iter,spearman_r,p_val
0,KB10_01_pursuitRoot.mat,spkTable_1,0,0.396992,0.083071
1,KB10_01_pursuitRoot.mat,spkTable_1,1,-0.052632,0.825581
2,KB10_01_pursuitRoot.mat,spkTable_1,2,0.019549,0.934803
3,KB10_01_pursuitRoot.mat,spkTable_1,3,0.001504,0.994980
4,KB10_01_pursuitRoot.mat,spkTable_1,4,0.267669,0.253892
...,...,...,...,...,...
124997,LP06_29_pursuitRoot.mat,spkTable_8,857,0.315275,0.175737
124998,LP06_29_pursuitRoot.mat,spkTable_9,857,0.285714,0.222035
124999,LP06_29_pursuitRoot.mat,spkTable_7,858,-0.283133,0.226424
125000,LP06_29_pursuitRoot.mat,spkTable_8,858,0.155179,0.513581


In [14]:
PPC_first_second_tuning

,sessFile,neuron,bootstrap_iter,spearman_r,p_val
0,KB09_12_pursuitRoot.mat,spkTable_1,0,0.431579,0.057425
1,KB09_12_pursuitRoot.mat,spkTable_3,0,0.049624,0.835414
2,KB09_12_pursuitRoot.mat,spkTable_1,1,0.112782,0.635917
3,KB09_12_pursuitRoot.mat,spkTable_3,1,0.046617,0.845270
4,KB09_12_pursuitRoot.mat,spkTable_1,2,0.136842,0.565096
...,...,...,...,...,...
348078,LP06_36_pursuitRoot.mat,spkTable_2,938,0.105719,0.657338
348079,LP06_36_pursuitRoot.mat,spkTable_1,939,0.030839,0.897307
348080,LP06_36_pursuitRoot.mat,spkTable_2,939,0.146351,0.538101
348081,LP06_36_pursuitRoot.mat,spkTable_1,940,0.482136,0.031334


In [15]:
PPC_odd_even_tuning

,sessFile,neuron,bootstrap_iter,spearman_r,p_val
0,KB09_12_pursuitRoot.mat,spkTable_1,0,-0.075188,0.752727
1,KB09_12_pursuitRoot.mat,spkTable_2,0,0.104183,0.662029
2,KB09_12_pursuitRoot.mat,spkTable_3,0,0.125611,0.597723
3,KB09_12_pursuitRoot.mat,spkTable_1,1,-0.105263,0.658728
4,KB09_12_pursuitRoot.mat,spkTable_2,1,0.118689,0.618213
...,...,...,...,...,...
330005,LP06_36_pursuitRoot.mat,spkTable_2,834,0.260150,0.267980
330006,LP06_36_pursuitRoot.mat,spkTable_1,835,-0.058647,0.805993
330007,LP06_36_pursuitRoot.mat,spkTable_2,835,0.298609,0.200950
330008,LP06_36_pursuitRoot.mat,spkTable_1,836,-0.566917,0.009146


In [25]:

def count_n(dataframe, spk_prefix="spkTable"):

    group = dataframe.groupby(["sessFile", "neuron"])

    total_neurons = group.size()

    return total_neurons

In [35]:
def count_neurons(dataframe, spk_prefix="spkTable"):

    # Create an empty dictionary to hold the counts for each session.
    session_neuron_counts = {}

    # Iterate over unique sessions.
    for sess in dataframe["sessFile"].unique():
        # Select only rows for that session.
        session = dataframe[dataframe["sessFile"] == sess]
        
        # Identify spkTable columns that are not entirely NaN for this session.
        spk_cols = [col for col in session.columns 
                    if col.startswith(spk_prefix) and not session[col].isna().all()
        ]
        
        # Store the count for this session.
        session_neuron_counts[sess] = len(spk_cols)

    # Sum the counts over all sessions
    total_neurons = sum(session_neuron_counts.values())

    return session_neuron_counts, total_neurons

In [33]:
count_neurons(RSC_cleaned)

({'KB10_01_pursuitRoot.mat': 1,
  'KB10_02_pursuitRoot.mat': 2,
  'KB10_03_pursuitRoot.mat': 1,
  'KB10_04_pursuitRoot.mat': 2,
  'KB10_06_pursuitRoot.mat': 2,
  'KB10_07_pursuitRoot.mat': 2,
  'KB10_08_pursuitRoot.mat': 1,
  'KB10_09_pursuitRoot.mat': 3,
  'KB10_10_pursuitRoot.mat': 5,
  'KB10_11_pursuitRoot.mat': 4,
  'KB10_13_pursuitRoot.mat': 2,
  'KB10_15_pursuitRoot.mat': 1,
  'KB10_17_pursuitRoot.mat': 1,
  'KB10_20_pursuitRoot.mat': 1,
  'KB20_02_pursuitRoot.mat': 3,
  'KB20_03_pursuitRoot.mat': 4,
  'KB20_04_pursuitRoot.mat': 2,
  'KB20_05_pursuitRoot.mat': 2,
  'KB20_06_pursuitRoot.mat': 2,
  'KB20_07_pursuitRoot.mat': 2,
  'KB20_09_pursuitRoot.mat': 2,
  'KB20_10_pursuitRoot.mat': 3,
  'KB20_11_pursuitRoot.mat': 1,
  'KB20_12_pursuitRoot.mat': 3,
  'KB20_13_pursuitRoot.mat': 1,
  'KB20_14_pursuitRoot.mat': 3,
  'KB20_15_pursuitRoot.mat': 3,
  'KB20_19_pursuitRoot.mat': 4,
  'KB20_21_pursuitRoot.mat': 2,
  'KB20_22_pursuitRoot.mat': 1,
  'KB20_24_pursuitRoot.mat': 4,
  'KB20_

In [32]:
count_neurons(PPC_cleaned)

({'KB09_12_pursuitRoot.mat': 3,
  'KB09_13_pursuitRoot.mat': 2,
  'KB09_17_pursuitRoot.mat': 6,
  'KB09_18_pursuitRoot.mat': 11,
  'KB09_19_pursuitRoot.mat': 6,
  'KB09_20_pursuitRoot.mat': 1,
  'KB09_22_pursuitRoot.mat': 1,
  'KB09_23_pursuitRoot.mat': 1,
  'KB09_24_pursuitRoot.mat': 1,
  'KB09_25_pursuitRoot.mat': 1,
  'KB09_26_pursuitRoot.mat': 1,
  'KB10_05_pursuitRoot.mat': 3,
  'KB10_07_pursuitRoot.mat': 1,
  'KB10_08_pursuitRoot.mat': 4,
  'KB19_03_pursuitRoot.mat': 4,
  'KB19_04_pursuitRoot.mat': 6,
  'KB19_05_pursuitRoot.mat': 9,
  'KB19_07_pursuitRoot.mat': 13,
  'KB19_08_pursuitRoot.mat': 13,
  'KB19_14_pursuitRoot.mat': 6,
  'KB19_15_pursuitRoot.mat': 9,
  'KB19_16_pursuitRoot.mat': 11,
  'KB19_17_pursuitRoot.mat': 10,
  'KB19_18_pursuitRoot.mat': 10,
  'KB19_19_pursuitRoot.mat': 10,
  'KB19_20_pursuitRoot.mat': 10,
  'KB19_21_pursuitRoot.mat': 10,
  'KB19_22_pursuitRoot.mat': 8,
  'KB19_23_pursuitRoot.mat': 8,
  'KB19_24_pursuitRoot.mat': 6,
  'KB19_25_pursuitRoot.mat': 8,

In [37]:
count_n(RSC_first_second_tuning)

sessFile                 neuron    
KB10_01_pursuitRoot.mat  spkTable_1    1001
KB10_02_pursuitRoot.mat  spkTable_1    1001
                         spkTable_2    1001
KB10_03_pursuitRoot.mat  spkTable_1    1001
KB10_04_pursuitRoot.mat  spkTable_1    1001
                                       ... 
LP06_18_pursuitRoot.mat  spkTable_6    1001
LP06_24_pursuitRoot.mat  spkTable_4     985
LP06_29_pursuitRoot.mat  spkTable_7     935
                         spkTable_8     935
                         spkTable_9     935
Length: 136, dtype: int64

In [38]:
count_n(RSC_odd_even_tuning)

sessFile                 neuron    
KB10_01_pursuitRoot.mat  spkTable_1    1001
KB10_02_pursuitRoot.mat  spkTable_1    1001
                         spkTable_2    1001
KB10_03_pursuitRoot.mat  spkTable_1    1001
KB10_04_pursuitRoot.mat  spkTable_1    1001
                                       ... 
LP06_18_pursuitRoot.mat  spkTable_6    1001
LP06_24_pursuitRoot.mat  spkTable_4     941
LP06_29_pursuitRoot.mat  spkTable_7     859
                         spkTable_8     859
                         spkTable_9     859
Length: 136, dtype: int64

In [27]:
count_n(PPC_first_second_tuning)

sessFile                 neuron    
KB09_12_pursuitRoot.mat  spkTable_1     861
                         spkTable_3     861
KB09_13_pursuitRoot.mat  spkTable_1     679
                         spkTable_2     679
KB09_17_pursuitRoot.mat  spkTable_1    1001
                                       ... 
LP06_30_pursuitRoot.mat  spkTable_6     681
LP06_34_pursuitRoot.mat  spkTable_1     665
                         spkTable_2     665
LP06_36_pursuitRoot.mat  spkTable_1     941
                         spkTable_2     941
Length: 363, dtype: int64

In [26]:
count_n(PPC_odd_even_tuning)

sessFile                 neuron    
KB09_12_pursuitRoot.mat  spkTable_1    573
                         spkTable_2    573
                         spkTable_3    573
KB09_13_pursuitRoot.mat  spkTable_1    611
                         spkTable_2    611
                                      ... 
LP06_30_pursuitRoot.mat  spkTable_6    527
LP06_34_pursuitRoot.mat  spkTable_1    599
                         spkTable_2    599
LP06_36_pursuitRoot.mat  spkTable_1    837
                         spkTable_2    837
Length: 366, dtype: int64

# Find cells with corr values over the 95th and 99th percentiles and compare them to the true tuning values.

In [ ]:
#plot null distribution of spearman r values with true, 95th, 99th percentile lines for all cells
pursuit.tuning.plot_null_dist(RSC_first_second_tuning)

In [ ]:
#ID significant cells above 95th and 99th corr values
RSC_first_second_cells_95 = pursuit.tuning.get_significant_cells(RSC_first_second_tuning, percentile=95)
RSC_first_second_cells_99 = pursuit.tuning.get_significant_cells(RSC_first_second_tuning, percentile=99)


In [ ]:
#plot null distribution of spearman r values with true, 95th, 99th percentile lines for significant cells
pursuit.tuning.plot_null_dist(RSC_first_second_tuning, neurons=RSC_first_second_cells_95)

In [ ]:
# plot heatmaps of significant cells and return two dataframes; one for peak-sorted neurons ready to plot and one with binned tuning values 
df_sorted , df_binned = pursuit.tuning.sig_cells_heatmap(RSC_cleaned, circle_boundaries, RSC_first_second_cells_95, percentile=95)

In [ ]:
#plot at total laser occupancy per bin per session (and save to pdf bc there's a lot of plots)

pursuit.tuning.occ_dist_to_pdf(df_binned)

In [ ]:
#plot individual significant cell tuning curves

pursuit.tuning.plot_sig_tuning_curves(RSC_cleaned, circle_boundaries, RSC_first_second_cells_95)

# ignore everything under here for now

# Investigating time shifts.

In [ ]:
def unique_time_entries(dataframe):
    all_sessions = []

    for sessFile in dataframe["sessFile"].unique():
        session = dataframe[dataframe["sessFile"] == sessFile]
        session_times = session["time"]

        times_unique = session_times.astype("float64").nunique()
        times_count = session_times.astype("float64").value_counts().to_dict()

        all_sessions.append({
            "sessFile": sessFile,
            "times_unique": times_unique,
            "unique_times_count": times_count
        })

    return pd.DataFrame(all_sessions)

In [ ]:
RSC_sessions_times = unique_time_entries(RSC_sessions)
RSC_sessions_times

In [ ]:
def inspect_time_counts(session_row):
    times= session_row["unique_times_count"]
    sorted_times = sorted(times.items())
    return pd.DataFrame(sorted_times, columns=["time", "count"])

In [ ]:
lp03_25 = RSC_sessions_times[RSC_sessions_times["sessFile"] == "LP03_25_pursuitRoot.mat"]

lp03_25_row = lp03_25.iloc[0]

lp03_25_inspection = inspect_time_counts(lp03_25_row)


lp03_25_inspection

# Find distance of laser points to boundary and bin data by distance.

In [ ]:
#find distance of normalized laser points to circle boundary by each session
#function takes the normalized laser/spikes and circle boundaries dataframes

RSC_laser_spks_bounds = pursuit.tuning.dist_to_bounds(RSC_laser_spks, circle_boundaries)
CA1_laser_spks_bounds = pursuit.tuning.dist_to_bounds(CA1_laser_spks, circle_boundaries)
CA3_laser_spks_bounds = pursuit.tuning.dist_to_bounds(CA3_laser_spks, circle_boundaries)

In [ ]:
CA3_laser_spks_bounds.head(50)

In [ ]:
#bin the data!
RSC_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(RSC_laser_spks_bounds)
CA1_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(CA1_laser_spks_bounds)
CA3_laser_spikes_binned = pursuit.tuning.bin_spikes_laser(CA3_laser_spks_bounds)

# Normalize spike counts, smooth data, peak sort neurons, and plot tuning curves.

In [ ]:
#calculate raw tuning curves
RSC_tuning = pursuit.tuning.calculate_tuning(RSC_laser_spikes_binned)
CA1_tuning = pursuit.tuning.calculate_tuning(CA1_laser_spikes_binned)
CA3_tuning = pursuit.tuning.calculate_tuning(CA3_laser_spikes_binned)

In [ ]:
#plot all neuron tuning curves for a sanity check

pursuit.tuning.plot_tuning_curves(CA3_tuning)

In [ ]:
# z-score and normalize data
RSC_z_scored = pursuit.tuning.z_score_norm(RSC_tuning)
CA1_z_scored = pursuit.tuning.z_score_norm(CA1_tuning)
CA3_z_scored = pursuit.tuning.z_score_norm(CA3_tuning)

In [ ]:
# apply smoothing and pivot data
RSC_smoothed = pursuit.tuning.pivot_smooth(RSC_z_scored)
CA1_smoothed = pursuit.tuning.pivot_smooth(CA1_z_scored)
CA3_smoothed = pursuit.tuning.pivot_smooth(CA3_z_scored)

In [ ]:
# peak sort the data
RSC_smoothed_sorted = pursuit.tuning.peak_sort(RSC_smoothed)
CA1_smoothed_sorted = pursuit.tuning.peak_sort(CA1_smoothed)
CA3_smoothed_sorted = pursuit.tuning.peak_sort(CA3_smoothed)

In [ ]:
#plot heatmaps
pursuit.tuning.heatmap(RSC_smoothed_sorted)
pursuit.tuning.heatmap(CA1_smoothed_sorted)
pursuit.tuning.heatmap(CA3_smoothed_sorted)

# Calculate neuron counts

In [ ]:
RSC_count = pursuit.tuning.count_neurons(RSC_cleaned)
CA1_count = pursuit.tuning.count_neurons(CA1_cleaned)
CA3_count = pursuit.tuning.count_neurons(CA3_cleaned)

In [ ]:
RSC_count


In [ ]:
CA1_count

In [ ]:
CA3_count